In [ ]:
from ipywidgets import interact
import matplotlib.pyplot as plt
import matplotlib.patches
import numpy as np
from pyprind import prog_percent
import random
from skimage.io import imread, imsave
from zebrafishframework import io
from zebrafishframework import rendering
from zebrafishframework import segmentation

from importlib import reload

In [ ]:
rendering = reload(rendering)
segmentation = reload(segmentation)

In [ ]:
!ls /Users/koesterlab/segmented/control/*.h5

In [ ]:
fish = 14
base = '/Users/koesterlab/segmented/control/fish%02d_6dpf_medium' % fish
roi_file = base + '_rois.npy'
traces_file = base + '_traces.npy'
rendering_file = base + '_rendering.mp4'
anatomy_file = base + '_std_dev.h5'

In [ ]:
rois = np.asarray([(b, a, c, d) for a, b, c, d in np.load(roi_file)]) # fixme!!!!!
traces_raw = np.load(traces_file)
anatomy = io.load(anatomy_file)[0]
mask = np.ones(anatomy.shape) # no mask

In [ ]:
ts = np.arange(traces_raw.shape[1])

In [ ]:
plt.figure(figsize=(12, 12))
plt.imshow(anatomy[10], aspect='auto', cmap=plt.get_cmap('Greys_r'))

In [ ]:
roi_map = segmentation.draw_rois(rois, anatomy)
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(roi_map[i])

In [ ]:
traces = traces_raw.copy()
F_max = np.percentile(traces_raw, 99.9)
print('F_max: %d' % F_max)
bounds = (97, F_max)

mean = np.mean(traces, axis=1)

# exclude rois with a too low mean value
valid_mask = np.full(mean.shape, True)
too_low = mean < bounds[0]

# clamp high values
traces = np.minimum(traces, bounds[1])
valid_mask[too_low] = False
print('Too low: %.1f%%' % (100*np.sum(too_low)/np.alen(mean)))

traces = traces[valid_mask]

In [ ]:
fig = plt.figure(figsize=(12, 8))
plt.imshow(traces[np.argsort(np.average(traces, axis=1))],aspect='auto')
plt.xlabel('t')
plt.ylabel('ROI ID')
plt.colorbar()

mean = np.mean(traces_raw, axis=1)
fig = plt.figure(figsize=(12, 6))
plt.hist(mean, bins=100);
plt.xlabel('average F')
plt.ylabel('#ROIs')
plt.title('F distribution (/w outliers)')

points = traces_raw.flatten()
fig = plt.figure(figsize=(12, 6))
plt.hist(points, bins=100);
plt.yscale('log')
plt.xlabel('F')
plt.ylabel('#(data_points)')
plt.title('value distribution')

In [ ]:
# correct bleaching
time_constant = -0.000065
factors = np.exp(-time_constant*ts)
F = traces*factors

plt.figure(figsize=(12, 8))
plt.plot(ts, np.mean(traces, axis=0), label='mean F')
plt.plot(ts, np.mean(F, axis=0), label='mean F (corrected)')
plt.legend(loc=4);

In [ ]:
# time from where to sample pre F
baseline = np.arange(50, 120)
dFF = segmentation.dFF(traces, baseline)

In [ ]:
np.min(dFF)

In [ ]:
dFF_shuffled = dFF.copy()
random.shuffle(dFF_shuffled)

plt.figure(figsize=(12, 8))
plt.plot(ts, traces[0], color='black', alpha=1)

In [ ]:
F = traces
mean_F = np.mean(F, axis=0)
mean_trace = np.mean(filtered_dFF, axis=0)

import scipy.stats
(a_s, b_s, r, tt, stderr) = scipy.stats.linregress(ts[:], np.log(mean_F[:]))
reg = np.exp(b_s+a_s*ts)

plt.figure(figsize=(12,10))
#plt.ylim((0, 350))
#plt.plot(ts, mean_F, 'blue')
#plt.plot(ts, reg, 'red')
#plt.plot(ts, mean_trace-.1*std_trace, 'red', alpha=.5)
#plt.plot(ts, mean_trace+.1*std_trace, 'red', alpha=.5)

# from some controls
a_s = -0.00015

factors = np.exp(-a_s*ts)

F_adjusted = F*factors
dFF_adjusted = segmentation.dFF(F_adjusted, np.arange(100, 160))

plt.plot(ts, np.mean(F, axis=0))

In [ ]:
a_s

In [ ]:
plt.figure(figsize=(12,10))
#plt.ylim((-1, 2))
plt.xlim((800, 1000))

ts = np.arange(F.shape[1])
choices = np.arange(np.alen(F))
random.shuffle(choices)
choices = choices[:100]

chosen = F[choices]

for trc in chosen:
    plt.plot(ts, trc, color='red', alpha=.05)
"""

for filt in filtered:
    plt.plot(ts, filt, color='black', alpha=.05)

"""

Render traces

In [ ]:
rendering = reload(rendering)
render_ts = np.arange(0, 1800, 1)

def color_func(dFF):
    final_a = (0, 255, 0)
    final_b = (255, 0, 255)
    alpha = 1
    max_dFF = 1
    c = np.array(final_b if dFF > 0 else final_a, dtype=np.float32)
    dFF = min(abs(dFF), max_dFF)/max_dFF
    return np.array(c*alpha*dFF, dtype=np.uint8)

activity = rendering.orthogonal(rois, filtered_dFF, color_func, render_ts, (1024, 1024))
plt.figure(figsize=(12,10))
plt.imshow(np.array(activity[0], dtype=np.uint8))


In [ ]:
rendering.to_file(base + '_rendering.mp4', activity)

In [ ]:
#plt.figure(figsize=(12,10))
def visualize_clustering(dFF, clustering):
    n = np.max(clustering) + 1
    cmap = plt.cm.get_cmap('hsv', n+1)
    color_func = lambda roi_id: np.array(cmap(km.labels_[roi_id])[:3])*255
    
    for cluster in range(n):
        fig = plt.figure(figsize=(12,5))
        cluster_traces = dFF[clustering == cluster]
        img = plt.imshow(cluster_traces[np.argsort(np.average(cluster_traces, axis=1))],aspect='auto', vmax=2)
        plt.title('cluster %d' % (cluster + 1), color=cmap(cluster))
        plt.colorbar()
    
    t = np.arange(traces_dFF.shape[1])
    """
    bins = np.arange(2, -1, .01)
    im = np.zeros((bins.shape[0], t.shape[0], 3))
    im.axes.yaxis.
    """

    plt.figure(figsize=(12,10))
    
    plt.ylim((-1, 2))
    for cluster in range(n):
        cluster_traces = dFF[clustering == cluster]
        random.shuffle(cluster_traces)
        for trace in cluster_traces[:200]:
            plt.plot(t, trace, color=cmap(cluster), alpha=.005)
    
    roi_map = segmentation.draw_rois(rois, anatomy_std, color_func=color_func)
    @interact
    def browse(i:(0,20)):
        plt.figure(figsize=(12,12))
        plt.imshow(roi_map[i])


Sorted dFF

In [ ]:
indices = np.argsort(np.average(filtered_dFF[:, 1600:], axis=1))

def map_to_group(index, n):
    percentile = index/n
    if percentile < .2:
        return 0
    if percentile > .8:
        return 2
    return 1

clustering = np.array(list(map(lambda e: map_to_group(e, filtered_dFF.shape[0]), np.argsort(indices))))

"""
plt.figure(figsize=(12,10))
plt.imshow(traces_dFF[indices],aspect='auto', vmax=2)
plt.colorbar()
"""



In [ ]:
%%time
visualize_clustering(filtered_dFF, clustering)

Clustered traces

In [ ]:
%%time
n = 10
from sklearn.cluster import KMeans
km = KMeans(n)
km.fit(filtered_dFF[:,1000:1600])

In [ ]:
%%time
visualize_clustering(filtered_dFF, km.labels_)

In [ ]:
roi_map = segmentation.draw_rois(rois, anatomy_std, color_func=color_func)
@interact
def browse(i:(0,20)):
    plt.figure(figsize=(12,12))
    plt.imshow(roi_map[i])

In [ ]:
mean_before = traces_dFF[:,100:160].mean(1)
mean_after  = traces_dFF[:,700:].mean(1)
bins = np.arange(-1, 2, .02)
before_hist = np.histogram(mean_before, bins=bins)
after_hist = np.histogram(mean_after, bins=bins)

In [ ]:
plt.figure(figsize=(12, 10))
#plt.hist(mean_before, bins, alpha=.5, label='before')
plt.hist(mean_after, bins, alpha=.5, label='after')
plt.legend(loc=0)
plt.show()

In [ ]:
traces = traces_raw.copy()

# lower outlier detection
#points = np.mean(traces, axis=1)
points = traces.flatten()

# calculate quartile function
points.sort()
x = np.arange(np.alen(points))/np.alen(points)
y = points

def differentiate(x, y):
    dy = np.zeros(y.shape, np.float)
    dy[0:-1] = np.diff(y)/np.diff(x)
    dy[-1] = (y[-1] - y[-2])/(x[-1] - x[-2])
    return dy

grad = differentiate(x, y)
mid = int(np.alen(grad)/2)

# find maxiumum gradient in first half as cut
arg = np.argmax(grad[:mid])+1
lower_percentile = x[arg]
lower_threshold = y[arg]

print('Lower %.1f%% are outliers (F < %d).' % (lower_percentile*100, threshold))

arg = np.min((np.argmax(grad[mid:])+mid-1, np.alen(x)-1))
upper_percentile = x[arg]
upper_threshold = y[arg]

print('Upper %.1f%% are outliers (F > %d).' % ((1-upper_percentile)*100, upper_threshold))

valid = np.array([True]*traces.shape[0])
mean = np.mean(traces, axis=1)
valid[mean < lower_threshold] = False
valid[np.any(traces > upper_threshold, axis=1)] = False

traces = traces[valid]